In [1]:
from datetime import datetime
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import json

In [5]:
today = datetime.today().strftime("%Y%m%d")
base_url = "https://news.naver.com/main/list.naver?mode=LSD&mid=shm&sid1=101&date=" + today

In [6]:
def get_news_links(page):
    url = base_url + "&page=" + str(page)
    response = requests.get(url)
    soup = BS(response.text, "html.parser")
    
    links = []
    for a in soup.select("ul.type06_headline li dl dt a"):
        links.append(a["href"])
    for a in soup.select("ul.type06 li dl dt a"):
        links.append(a["href"])
    
    return links

def get_news_content(url):
    response = requests.get(url)
    soup = BS(response.text, "html.parser")
    
    title_tag = soup.select_one("h2.media_end_head_headline")
    content_tag = soup.find('article',{'id':'dic_area'})
    
    if title_tag and content_tag:
        title = title_tag.get_text().strip()
        content = content_tag.get_text().strip()
        return title, content
    else:
        return None, None

In [11]:
# 크롤링 시작
news_links = []
page = 1

while True:
    links = get_news_links(page)
    if not links or any(link in news_links for link in links):
        break
    news_links.extend(links)
    page += 1


In [12]:
# 뉴스 기사 내용 크롤링
news_contents = []
for link in news_links[:20]:    # 숫자 변경
    try:
        title, content = get_news_content(link)
        news_contents.append((title, content))
    except Exception as e:
        print(f"Failed to get content from {link}: {e}")

In [2]:
import numpy as np
import itertools

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer # pip install sentence_transformers
from bareunpy import Tagger # pip install bareunpy

tagger = Tagger('koba-Q2CYNCI-XZ7E7PI-X6YRKPY-K4Z2KMY')
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')


/home/hadoop/project/third_project/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [39]:
def keyword_ext(text):

    tokenized_doc = tagger.pos(text)
    tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'NNG' or word[1] == 'NNP'])

    n_gram_range = (1,1)

    count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
    candidates = count.get_feature_names_out()

    doc_embedding = model.encode([text])
    candidate_embeddings = model.encode(candidates)

    return mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2)

def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    # print(keywords_idx)

    return [words[idx] for idx in keywords_idx]


In [32]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from bareunpy import Tagger
from datetime import datetime

def keyword_ext(text):
    """텍스트에서 키워드를 추출합니다."""
    tokenized_doc = tagger.pos(text)
    tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'NNG' or word[1] == 'NNP'])
    if not tokenized_nouns.strip():
        return []
    
    count = CountVectorizer(ngram_range=(1, 1)).fit([tokenized_nouns])
    candidates = count.get_feature_names_out()

    if len(candidates) == 0:
        return []

    doc_embedding = model.encode([text])
    candidate_embeddings = model.encode(candidates)

    return mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2)

def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):
    """다양성을 고려한 최대 마진 적중률(MMR)을 계산하여 키워드를 추출합니다."""
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)
    word_similarity = cosine_similarity(candidate_embeddings)

    if len(word_doc_similarity) == 0 or len(word_similarity) == 0:
        return []

    keywords_idx = [np.argmax(word_doc_similarity)]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)
        mmr = (1 - diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)

        if mmr.size == 0:
            break

        mmr_idx = candidates_idx[np.argmax(mmr)]
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

# 뉴스 내용을 포함한 CSV 파일 읽기
today = datetime.today().strftime("%Y%m%d")
news_contents = pd.read_csv(f"{today}.csv", encoding='utf-8')
news_contents = news_contents.dropna()

# 제목과 키워드를 저장할 DataFrame 생성
df = pd.DataFrame(columns=["Title", "Keywords", "Keyword_Count"])

# 키워드 추출 및 DataFrame에 저장
for i, row in news_contents.iterrows():
    title = row['Title']
    content = row['content']
    if content:
        keywords = keyword_ext(content)
        new_row = pd.DataFrame({"Title": [title], "Keywords": [keywords], "Keyword_Count": [len(keywords)]})
        df = pd.concat([df, new_row], ignore_index=True)

# DataFrame 출력
print(df)

# 결과를 CSV 파일로 저장
df.to_csv(f"{today}_keywords1.csv", index=False, encoding='utf-8-sig')

# 키워드와 그 빈도를 저장할 DataFrame 생성
all_keywords = []

for keywords in df['Keywords']:
    all_keywords.extend(keywords)

keyword_counts = pd.Series(all_keywords).value_counts().reset_index()
keyword_counts.columns = ['Keywords', 'Count']

# 최종 키워드 빈도수 DataFrame 출력
print(keyword_counts)

# 결과를 CSV 파일로 저장
keyword_counts.to_csv(f"{today}_keyword_counts1.csv", index=False, encoding='utf-8-sig')

                                                  Title  \
0                         韓 기업 발목 잡는 상속세… “자본이득세로 전환해야”   
1                          SK온 "흑자전환까지 임원연봉 동결" 비상경영 돌입   
2                  `영업이익률 66%` 그 회사…김형태 시프트업 대표, 1조 부호로   
3                              [속보] 코스피, 종가 기준 2800선 탈환   
4     정태영 부회장 “해외 진출 초석 쌓는다”…비자 CEO·글로벌 금융사 CEO들 현대카...   
...                                                 ...   
3951                      GE헬스케어 코리아, 김용덕 신임 대표이사 사장 선임   
3952                       화성산업 'HS화성'으로 이름 바꾸고 새 출발 선언   
3953                 신한·토스證, 클라우드 기반 오픈플랫폼 추진 위한 MOU 체결   
3954                 현대차, 인도네시아서 '현지 배터리' 탑재 코나 일렉트릭 생산   
3955        최태원, 美 출장서 아마존·인텔 CEO 잇따라 만나…"AI 보폭 맞춰 뛰어야"   

                                     Keywords Keyword_Count  
0           [한국조세정책학회장, 경제, 토론자, 한국경제연구원, 미래]             5  
1                       [임원, 효율, 회의, 경영자, 조직]             5  
2                    [인수, 자금, nh투자증권, 주식, 가치]             5  
3                 [지난달, 포인트, 거래, 연합뉴스코스피, 지